In [16]:
# Make all the imports in this cell.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
from tqdm import tqdm

In [17]:
# Fetch the data from provided files.
data = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/data.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv", index_col="row-col")

Make the imputations and make sure that there are no non-imputated values left.

In [38]:
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='mean')
imputed_data = data
imputed_data[:] = imp.fit_transform(data)
print("Are all values imputed? " + str(imputed_data.isnull().sum().iloc[0] == 0))

Are all values imputed? True


Calculate the score of the dataset.

In [39]:
# Nothing yet.

Turn the imputed values into a suitable form, format the values to be submitted and create the submission file.

In [40]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = data.loc[row, col]

submission.to_csv('mean_benchmark.csv')

100%|██████████| 1000000/1000000 [01:45<00:00, 9514.31it/s]
